# AI Code Commenting Assistant  

## Project Summary  

**Purpose**:  
An AI-powered assistant that automatically generates **clear, concise code comments** to improve code readability and maintainability.  

**Key Features**:  
- **Language-Agnostic**: Auto-detects programming languages or allows manual specification  
- **Smart Commenting**: Focuses on explaining **complex logic, algorithms, and edge cases** (not obvious syntax)  
- **Customizable**: Optional focus areas let users prioritize specific parts (e.g., database queries, recursion)  
- **Efficient Workflow**: Processes code in chunks and preserves original formatting  

**Benefits**:  
✔ Saves time writing documentation  
✔ Helps developers understand unfamiliar code  
✔ Supports multiple languages (Python, JavaScript, C++, SQL, etc.)  
✔ Avoids redundant comments on trivial operations  

**Example Use Case**:  
```python  
# Before AI:  
def fib(n):  
    if n <= 1: return n  
    else: return fib(n-1) + fib(n-2)  

# After AI:  
def fib(n):  
    # Recursively computes nth Fibonacci number (O(2^n) time)  
    if n <= 1: return n  # Base case  
    else: return fib(n-1) + fib(n-2)  # Recursive case  

In [1]:
!pip install --upgrade huggingface_hub

In [2]:
import os
import io
from dotenv import load_dotenv
import google.generativeai as genai
from google.generativeai import types
from openai import OpenAI
from anthropic import Anthropic
from huggingface_hub import InferenceClient
import gradio as gr

In [3]:
# load environments
load_dotenv(override=True)
os.environ['ANTHROPIC_API_KEY'] = os.getenv("ANTHROPIC_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
gemini_api_key= os.getenv("GOOGLE_API_KEY")

In [4]:
#initialize remote models
claude= Anthropic()
gemini = genai.Client(api_key=gemini_api_key)

#opensource models
qwen = InferenceClient(provider="featherless-ai")

AttributeError: module 'google.generativeai' has no attribute 'Client'

In [ ]:
#initialize local model
llama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
#models
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-pro"
qwen_model= "Qwen/Qwen2.5-Coder-32B-Instruct"
llama_model = "llama3:8b"

In [ ]:
system_message="""
You are an expert AI specialized in code documentation. Your task is to generate concise, meaningful comments that explain the purpose and logic of provided code. Follow these rules:

1. **Infer language**: Auto-detect programming language and use appropriate comment syntax
2. **Explain why, not what**: Focus on purpose, edge cases, and non-obvious logic
3. **Be concise**: Maximum 1-2 sentences per comment block
4. **Prioritize key sections**: Only comment complex logic, algorithms, or critical operations
5. **Maintain structure**: Preserve original code formatting and indentation
6. **Output format**: Return ONLY commented code with no additional text

Commenting guidelines by language:
- Python: `# Inline comments` and `""Docstrings""`
- JavaScript/Java: `// Line comments` and `/* Block comments */`
- C/C++: `//` and `/* */`
- SQL: `-- Line comments`
"""

In [ ]:
def user_prompt(code):
    prompt = f"""
     i want to document my code for better understanding. Please generate meaningful necessary comments
     here is my code:
     {code}

     Return ONLY commented code with no additional text
     """

    return prompt

In [ ]:
def conversation_template(code):
    messages = [
        {"role":"system", "content":system_message},
        {"role":"user","content":user_prompt(code)}
    ]
    return messages

In [ ]:
def stream_gemini(code):
    message = user_prompt(code)
    response = gemini.models.generate_content_stream(
        model=gemini_model,
        config= types.GenerateContentConfig(
            system_instruction = system_message,
            temperature = 0.8,
        ),
        contents = [message]
    )

    result = ""
    for chunk in response:
        result += chunk.text or ""
        yield result

In [ ]:
def stream_claude(code):
    messages = [{"role":"user","content":user_prompt(code)}]
    response = claude.messages.stream(
        model= claude_model,
        temperature=0.8,
        messages = messages,
        max_tokens=5000
    )

    result = ""
    with response as stream:
        for text in stream.text_stream:
            result += text or ""
            yield result
    

In [ ]:
def stream_opensource(code,model):
    model = model.lower()
    client = globals()[model]
    model = globals()[f"{model}_model"]
    stream = client.chat.completions.create(
        model = model,
        messages= conversation_template(code),
        temperature = 0.7,
        stream = True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def commentor(code, model):
    model =model.lower()
    if model == "claude":
        result = stream_claude(code)
    elif model == "gemini":
        result = stream_gemini(code)
    elif model == "qwen" or model == "llama":
        result = stream_opensource(code, model)


    for code in result:
        yield code.replace("```cpp\n","").replace("```python\n","").replace("```javascript\n","").replace("```typescript\n","").replace("```","")

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("# Genarate comment")
    with gr.Row():
        raw_code = gr.Textbox(label="Raw Code:", lines=10)
        commented_code = gr.Textbox(label="Commented_code",lines=10)
    with gr.Row():
        models = gr.Dropdown(["Gemini","Claude","Llama","Qwen"], value="Gemini")
    with gr.Row():
        generate_comment = gr.Button("Generate Comment")

    generate_comment.click(commentor, inputs=[raw_code, models], outputs=[commented_code])

In [ ]:
ui.launch(inbrowser=True,debug=True)